<a href="https://colab.research.google.com/github/gheesung/tfnlearn-edgetpu-classify/blob/master/Retraining_Image_Classification_in_Colab_for_Edge_TPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Learning (Image Classification) in Colab for Edge TPU

##Background
Google provides a tutorial for  [transfer learning](https://coral.withgoogle.com/docs/edgetpu/retrain-classification/) on a local machine for Edge TPU. If you do not have a GPU, this is going to be quite slow.

Colab is an excellence free tool provided by Google which provides free GPU. This tutorial shows how to use Colab to perform the transfer learning and convert the pb file to tflite. The tflite file is downloaded to the local machine and compile the model for the Edge TPU.

This tutorial is adapted from [tensorflow image retraining example](https://www.tensorflow.org/hub/tutorials/image_retraining)

###Download the flower photos from tensorflow site

In [0]:
%cd /content
!curl -LO http://download.tensorflow.org/example_images/flower_photos.tgz
!tar xzf flower_photos.tgz

/content
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  218M  100  218M    0     0  52.1M      0  0:00:04  0:00:04 --:--:-- 52.1M


###Get the image retraining and label scripts

In [0]:
%cd /content
!mkdir example_code
%cd example_code
!curl -LO https://github.com/tensorflow/hub/raw/master/examples/image_retraining/retrain.py
!curl -LO https://github.com/tensorflow/tensorflow/raw/master/tensorflow/examples/label_image/label_image.py

/content
/content/example_code
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   158  100   158    0     0    440      0 --:--:-- --:--:-- --:--:--   440
100 55834  100 55834    0     0  79762      0 --:--:-- --:--:-- --:--:-- 25.7M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   175  100   175    0     0    491      0 --:--:-- --:--:-- --:--:--   491
100  4727  100  4727    0     0   6752      0 --:--:-- --:--:-- --:--:--  6752


###Retraining of the model
For this tutorial we are using Mobilenet V1 and the model is available for download from [tfhub](https://tfhub.dev/s).

Since we are using edge TPU, the model requires a quantised model. For this tutorial we have selected the [quantised model of mobilenet v1](https://tfhub.dev/google/imagenet/mobilenet_v1_100_224/quantops/feature_vector/3) for retraining.

retrain.py allows various hyper parameters to be changed.

--learning_rate

--how_many_training_steps

In [0]:
# quantized mode
%cd /content/example_code
!rm -rf /tmp/*
!python retrain.py --image_dir /content/flower_photos \
    --tfhub_module https://tfhub.dev/google/imagenet/mobilenet_v1_100_224/quantops/feature_vector/3 


/content/example_code
W0713 09:52:11.507035 139795277698944 deprecation_wrapper.py:119] From retrain.py:1349: The name tf.app.run is deprecated. Please use tf.compat.v1.app.run instead.

W0713 09:52:11.507595 139795277698944 deprecation_wrapper.py:119] From retrain.py:976: The name tf.logging.FATAL is deprecated. Please use tf.compat.v1.logging.FATAL instead.

W0713 09:52:11.507727 139795277698944 deprecation_wrapper.py:119] From retrain.py:977: The name tf.logging.ERROR is deprecated. Please use tf.compat.v1.logging.ERROR instead.

W0713 09:52:11.507842 139795277698944 deprecation_wrapper.py:119] From retrain.py:978: The name tf.logging.WARN is deprecated. Please use tf.compat.v1.logging.WARN instead.

W0713 09:52:11.507934 139795277698944 deprecation_wrapper.py:119] From retrain.py:979: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.

W0713 09:52:11.508023 139795277698944 deprecation_wrapper.py:119] From retrain.py:980: The name tf.logging.DEBUG 

In [0]:
# test a sample to check if the retraining works
!python label_image.py \
--graph=/tmp/output_graph.pb --labels=/tmp/output_labels.txt \
--input_layer=Placeholder \
--output_layer=final_result \
--input_height=224 --input_width=224 \
--image=/content/flower_photos/sunflowers/1008566138_6927679c8a.jpg

W0713 09:57:35.667819 140180303562624 deprecation_wrapper.py:119] From label_image.py:28: The name tf.GraphDef is deprecated. Please use tf.compat.v1.GraphDef instead.

W0713 09:57:35.797549 140180303562624 deprecation_wrapper.py:119] From label_image.py:45: The name tf.read_file is deprecated. Please use tf.io.read_file instead.

W0713 09:57:35.801704 140180303562624 deprecation_wrapper.py:119] From label_image.py:59: The name tf.image.resize_bilinear is deprecated. Please use tf.compat.v1.image.resize_bilinear instead.

2019-07-13 09:57:35.806277: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcuda.so.1
2019-07-13 09:57:35.854537: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1005] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-07-13 09:57:35.854963: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1640] Found device 0 with prope

###Convert the pb model to tflite files
After this step, the file is downloaded to the local machine where the Coral Edge TPU is installed. 

The tflite file is to be compiled to Edge TPU model using the steps in this [tutorial](https://coral.withgoogle.com/docs/edgetpu/retrain-classification/).

edgetpu_compiler filegeneratedbystepbelow.tflite

In [0]:
!tflite_convert \
  --output_file=/tmp/output_graph.tflite \
  --graph_def_file=/tmp/output_graph.pb \
  --inference_type=QUANTIZED_UINT8 \
  --input_shapes=1,224,224,3 \
  --input_arrays=Placeholder \
  --output_arrays=final_result \
  --mean_values=128 \
  --std_dev_values=127


2019-07-13 09:57:51.110430: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcuda.so.1
2019-07-13 09:57:51.161410: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1005] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-07-13 09:57:51.161897: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1640] Found device 0 with properties: 
name: Tesla T4 major: 7 minor: 5 memoryClockRate(GHz): 1.59
pciBusID: 0000:00:04.0
2019-07-13 09:57:51.162189: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcudart.so.10.0
2019-07-13 09:57:51.163276: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcublas.so.10.0
2019-07-13 09:57:51.164412: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcufft.so.10.0
